In [ ]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from spacy import displacy
import edsnlp, edsnlp.pipes as eds

input_dir = "../data_clean/"
visuals_dir = "../visuals/" 

import random

from IPython.display import clear_output
import random

In [ ]:
assert os.path.exists(input_dir), f"Il faut exéctuer le script de nettoyage avant de lancer ce notebook ({input_dir} n'existe pas)."

df_person = pd.read_pickle(os.path.join(input_dir, 'df_person.pkl'))
df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
df_visit = pd.read_pickle(os.path.join(input_dir, 'df_visit.pkl'))
df_condition = pd.read_pickle(os.path.join(input_dir, 'df_condition.pkl'))

df_facteur_risque = pd.read_pickle(os.path.join(input_dir, 'df_facteur_risque.pkl'))

In [12]:
from IPython.display import clear_output

total = 0

faux_positif = 0
faux_negatif = 0
fumeurs = 0
non_fumeurs = 0

true_positif = 0
true_negatif = 0

n = 900

echantillons = [random.randint(0, len(df_person) - 1) for _ in range(n)]

for i in echantillons:
    person = df_person.iloc[i]

    try:
        person_id = person['person_id']
        visit = df_visit[df_visit['person_id'] == person_id].iloc[0]
        note = df_note[df_note['visit_occurrence_id'] == visit['visit_occurrence_id']].iloc[0]
    except IndexError:
        print("No visit or note found for this person.")
        continue
    print(note['note_text'])

    res = input("Le patient est-il un fumeur? (y/n): ")
    total += 1

    fumeur = df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id]['fumeur'].values

    if res == 'y':
        if fumeur:
            true_positif += 1
        else:
            faux_negatif += 1
    elif res == 'n':
        if fumeur:
            faux_positif += 1
        else:
            true_negatif += 1
    else:
        total -= 1

    clear_output(wait=True)

    print(f"Total: {total}, Faux Positifs: {faux_positif}, Faux Négatifs: {faux_negatif}, True Positifs: {true_positif}, True Négatifs: {true_negatif}")
    if res == 'exit':
        break

with open(os.path.join(input_dir, 'fumeur_results.txt'), 'w') as f:
    f.write(f"Total: {total}\n")
    f.write(f"Faux Positifs: {faux_positif}\n")
    f.write(f"Faux Négatifs: {faux_negatif}\n")
    f.write(f"True Positifs: {true_positif}\n")
    f.write(f"True Négatifs: {true_negatif}\n")

Total: 123, Faux Positifs: 5, Faux Négatifs: 3, True Positifs: 7, True Négatifs: 108


In [ ]:
from IPython.display import clear_output
import random
import os

total = 0

faux_positif_alcool = 0
faux_negatif_alcool = 0
buveurs = 0
non_buveurs = 0

n = 900

# On prélève n indices aléatoires dans df_person
echantillons = [random.randint(0, len(df_person) - 1) for _ in range(n)]

for i in echantillons:
    person = df_person.iloc[i]

    try:
        person_id = person['person_id']
        visit = df_visit[df_visit['person_id'] == person_id].iloc[0]
        note = df_note[df_note['visit_occurrence_id'] == visit['visit_occurrence_id']].iloc[0]
    except IndexError:
        print("Aucune visite ou note trouvée pour cette personne.")
        continue

    alcoolique = df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id]['alcool'].values

    if not alcoolique:
        continue # On ne considère que les patients avec un facteur de risque alcool, car il y en a trop peu

    print(note['note_text'])
    res = input("Le patient consomme-t-il de l'alcool ? (y/n) : ")

    total += 1

    

    if res == 'y':
        buveurs += 1
    elif res == 'n':
        faux_positif_alcool += 1
        buveurs += 1
    else:
        total -= 1

    clear_output(wait=True)

    print(f"Total : {total}, Faux positifs (alcool) : {faux_positif_alcool}, "
          f"Faux négatifs (alcool) : {faux_negatif_alcool}, Buveurs : {buveurs}, Non-buveurs : {non_buveurs}")

    if res == 'exit':
        break

# Enregistrement des résultats dans un fichier
if total > 10:
    with open(os.path.join(input_dir, 'alcool_results.txt'), 'w') as f:
        f.write(f"Total : {total}\n")
        f.write(f"Faux positifs (alcool) : {faux_positif_alcool}\n")
        f.write(f"Faux négatifs (alcool) : {faux_negatif_alcool}\n")
        f.write(f"Buveurs : {buveurs}\n")
        f.write(f"Non-buveurs : {non_buveurs}\n")
else:
    print("Pas assez de données pour enregistrer les résultats.")


Total : 26, Faux positifs (alcool) : 26, Faux négatifs (alcool) : 0, Buveurs : 26, Non-buveurs : 0
